## Import modules, save data to Pandas dataframe

In [1]:
from __future__ import division, print_function

from modules import PerformParser as parser
import numpy as np
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os
np.set_printoptions(precision=2)

In [2]:
init_notebook_mode(connected=True)

# Lets look at the variables stored in our dataframe

In [3]:
# read dict file, save as pandas DF
#datafile = parser.dictFileToDataFrame("data/2017-4-14-18-36/exp_data-2017-4-14-18-36.dict")
#datafile = parser.dictFileToDataFrame("data/2016-4-19-14-4/exp_data-2016-4-19-14-4.dict")
#datafile = parser.dictFileToDataFrame("data/2016-4-29-11-56/exp_data-2016-4-29-11-56.dict")
#datafile = parser.readPerformDict("data/2016-4-29-11-56/exp_data-2016-4-29-11-56.dict")

datafile = pd.read_pickle('data/calibration.pickle')

In [4]:
list(datafile.columns)

['viewPos_XYZ',
 'viewMat_4x4',
 'trialNumber',
 'blockNumber',
 'IOD',
 'IPD',
 'ballPos_XYZ',
 'cycEyeInHead_XYZ',
 'leftEyeInHead_XYZ',
 'rightEyeInHead_XYZ',
 'leftEyePos_XYZ',
 'rightEyePos_XYZ',
 'calibrationPos_XYZ',
 'calibrationCounter',
 'eyeTimeStamp',
 'frameTime',
 'eventFlag']

# EIH to GIW

In [5]:
def eihToGIW(rowIn):
    rowIn

    # Grab gransformation matrix
    headTransform_4x4 = np.reshape(rowIn["viewMat_4x4"],[4,4])

    # Transpose
    headTransform_4x4 = headTransform_4x4

    # Grab cyc EIH direction
    cycEyeInHead_XYZ = rowIn['cycEyeInHead_XYZ']
    # Add a 1 to convert to homogeneous coordinates
    cycEyeInHead_XYZW = np.hstack( [cycEyeInHead_XYZ,1])

    # Take the dot product!
    cycGIW_XYZW = np.dot( headTransform_4x4,cycEyeInHead_XYZW)
    
    # You must return as a list or a tuple
    return list(cycGIW_XYZW[0:3])

In [6]:
datafile['cycGIW_XYZ'] = datafile.apply(lambda row: eihToGIW(row),axis=1)

# Plot a calibration point, gaze vector, etc

Convert the datafram data into an array of arrays.
For you matlab users, this is your friend.

In [7]:
viewPos_row_XYZ = np.array(map(np.array,datafile['viewPos_XYZ']))

Now you can use matlab-like indexing
e.g. For all rows, get all X values

In [8]:
viewPos_row_XYZ[:,0]

array([-0.99, -1.  , -1.  , ..., -0.97, -0.97, -0.97])

In [15]:
def drawVector(row):

    #viewPos_XYZ = row['viewPos_XYZ']
    eihDir_xyz = row['cycEyeInHead_XYZ']
    #cycGIWEndpoint_XYZ = np.add(viewPos_XYZ, + np.multiply(cycGIW_XYZ,5))

    calibrationPos_XYZ = row['calibrationPos_XYZ']
    
    eihDir = go.Scatter3d(x=[0,eihDir_xyz[0]*5],
        y=[0,eihDir_xyz[2]*5],
        z=[0,eihDir_xyz[1]*5],
        mode='lines',
        line = dict(
          color=('rgb(20, 0, 145)'),
          width = 2)
        )
    
    headPos = go.Scatter3d(x=[0],
                           y=[0],
                           z=[0],
                           mode='markers',
                           marker={'color': '#48186a', 'size': 10},
                          )
    
    calibPos = go.Scatter3d(x=[calibrationPos_XYZ[0]],
                           y=[calibrationPos_XYZ[2]],
                           z=[calibrationPos_XYZ[1]],
                           mode='markers',
                           marker={'color': 'rgb(0,144,144)', 'size': 5},
                          )
    
    return([headPos,eihDir,calibPos])

In [27]:
gbTrial = datafile.groupby('trialNumber')
np.sort(gbTrial.groups.keys())


array([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010,
       1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021,
       1022, 1023, 1024, 1025])

In [28]:

gbTrial = datafile.groupby('trialNumber')

calibPt = gbTrial.get_group(1002)
traces = np.array(calibPt1.apply(lambda row: drawVector(row),axis=1))
traces = [item for sublist in traces for item in sublist]

width = 800
height = 800
xRange = [-10,10]
yRange = [-10,10]
zRange = [-10,10]

layout = go.Layout(title="Gaze in World", 
                width=width,
                height=height,
                showlegend=False,
                scene=go.Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=xRange, title='x Axis'),
                            yaxis=dict(range=yRange, title='y Axis'),
                            zaxis=dict(range=zRange, title='z Axis'),

                           ),
                margin=go.Margin(t=100),
                hovermode='closest',

                )


fig=go.Figure(data=list(traces),layout=layout)


iplot(fig)